In [27]:
# Import
import yfinance as yf
import pandas as pd
import numpy as np
import requests
import json
import talib as talib
from matplotlib import pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc
from matplotlib.pylab import date2num
from keras.models import Sequential
from keras.layers import Dense,LSTM,BatchNormalization,Flatten

In [28]:
# import list of top 100 stocks USA
# import stock list from CSV file
importedCsv = pd.read_csv (r'C:\Users\Eric\Desktop/StockList.csv', sep=',')
#print(importedCsv["Symbol"])

In [29]:
# Functions
def checkNonesString(df):
    # CHECK Object column for "None" strings.
    print(df[df['Symbol'].str.contains('None')].any())
    

def checkNans(df):
    # Check if there are NaNs
    print("-----IS NA per COL-------")
    print("Doesn't seem to be working")
    print(df[df.isna().any(axis=1)].sum())

    print("-----IS NA per COL-------")
    print(df.isna().sum())

    print("------printing sum of Nans for each column------")
    for col in df.columns:
        print(col + ": " + str(df[col].isnull().sum()))

def is_even(number):
    """Return whether an integer is even or not."""
    return 5      
        
def checkInfs(df):
    print("-------total amount of Infs per Column-----------")
    print(df.groupby(np.isinf(df["Open"])).count())

    print("-------total amount of Infs-----------")
    count = np.isinf(df["Open"]).values.sum()
    print("It contains " + str(count) + " infinite values")


def printDataframe(df):
    # Prints
    print("-----DATAFRAME-------")
    print(df.head(10))

    print("-----SHAPE-------")
    print(df.shape)

    print("-----COLUMNS-------")
    print(df.columns)

    print("-----DATATYPES-------")
    print(df.dtypes)



In [30]:
# Add historical data for all stocks in ImportedCSV file = 100 stocks
final_df = pd.DataFrame()

oneStockAAPL = yf.Ticker("AAPL").history(period="max")
# adding symbol for traceability
oneStockAAPL["Symbol"] = "AAPL"
# Rearragning the date column to a functionalt "timestamp"
#oneStockAAPL["Timestamp"] = oneStockAAPL.index
oneStockAAPL.insert(0, 'Timestamp', oneStockAAPL.index)
print(oneStockAAPL)
for col in importedCsv["Symbol"]:
    
    #add code for all stocks to ahve the same date interval
    
    #fetch data for every stock
    temp = yf.Ticker(col).history(period="max")
    # adding symbol for traceability
    temp["Symbol"] = col
    # Rearragning the date column to a functionalt "timestamp"
    temp["Timestamp"] = temp.index
    # concateing with the dataframe for all stocks
    final_df = pd.concat([final_df, temp], axis=0)

    

            Timestamp        Open        High         Low       Close  \
Date                                                                    
1980-12-12 1980-12-12    0.100453    0.100890    0.100453    0.100453   
1980-12-15 1980-12-15    0.095649    0.095649    0.095213    0.095213   
1980-12-16 1980-12-16    0.088661    0.088661    0.088224    0.088224   
1980-12-17 1980-12-17    0.090408    0.090845    0.090408    0.090408   
1980-12-18 1980-12-18    0.093029    0.093466    0.093029    0.093029   
...               ...         ...         ...         ...         ...   
2022-01-10 2022-01-10  169.080002  172.500000  168.169998  172.190002   
2022-01-11 2022-01-11  172.320007  175.179993  170.820007  175.080002   
2022-01-12 2022-01-12  176.119995  177.179993  174.820007  175.529999   
2022-01-13 2022-01-13  175.779999  176.619995  171.789993  172.190002   
2022-01-14 2022-01-14  171.339996  173.779999  171.089996  173.070007   

               Volume  Dividends  Stock Splits Sym

In [31]:
# restting index
oneStockAAPL.reset_index(drop=True, inplace=True)
final_df.reset_index(drop=True, inplace=True)

In [32]:
# Checking functions
printDataframe(final_df)
checkInfs(final_df)
checkNans(final_df)
checkNonesString(final_df)

-----DATAFRAME-------
       Open      High       Low     Close       Volume  Dividends  \
0  0.100453  0.100890  0.100453  0.100453  469033600.0        0.0   
1  0.095649  0.095649  0.095213  0.095213  175884800.0        0.0   
2  0.088661  0.088661  0.088224  0.088224  105728000.0        0.0   
3  0.090408  0.090845  0.090408  0.090408   86441600.0        0.0   
4  0.093029  0.093466  0.093029  0.093029   73449600.0        0.0   
5  0.098707  0.099143  0.098707  0.098707   48630400.0        0.0   
6  0.103511  0.103948  0.103511  0.103511   37363200.0        0.0   
7  0.107879  0.108315  0.107879  0.107879   46950400.0        0.0   
8  0.113556  0.113993  0.113556  0.113556   48003200.0        0.0   
9  0.124038  0.124475  0.124038  0.124038   55574400.0        0.0   

   Stock Splits Symbol  Timestamp  
0           0.0   AAPL 1980-12-12  
1           0.0   AAPL 1980-12-15  
2           0.0   AAPL 1980-12-16  
3           0.0   AAPL 1980-12-17  
4           0.0   AAPL 1980-12-18  
5 

C:\Users\Eric\AppData\Local\Temp/ipykernel_9680/3402867688.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(df[df.isna().any(axis=1)].sum())


Open: 1
High: 1
Low: 1
Close: 1
Volume: 1
Dividends: 0
Stock Splits: 0
Symbol: 0
Timestamp: 0
Open            False
High            False
Low             False
Close           False
Volume          False
Dividends       False
Stock Splits    False
Symbol          False
Timestamp       False
dtype: bool


In [33]:
# Finds the row witht the NaNs
#df1 = final_df[final_df['Symbol'].str.contains("NVDA")]
#is_NaN = df1.isnull()
#row_has_NaN = is_NaN.any(axis=1)
#rows_with_NaN = df1[row_has_NaN]
#print(rows_with_NaN)
#print(df1)

#print("--------")
#df2 = final_df.loc[46545:46550]
#print(df2)

# Dropping rows with NaNs, (should only be one in NVDA)
final_df.dropna(inplace=True)

In [34]:
# Checking functions
printDataframe(final_df)
checkInfs(final_df)
checkNans(final_df)
checkNonesString(final_df)

-----DATAFRAME-------
       Open      High       Low     Close       Volume  Dividends  \
0  0.100453  0.100890  0.100453  0.100453  469033600.0        0.0   
1  0.095649  0.095649  0.095213  0.095213  175884800.0        0.0   
2  0.088661  0.088661  0.088224  0.088224  105728000.0        0.0   
3  0.090408  0.090845  0.090408  0.090408   86441600.0        0.0   
4  0.093029  0.093466  0.093029  0.093029   73449600.0        0.0   
5  0.098707  0.099143  0.098707  0.098707   48630400.0        0.0   
6  0.103511  0.103948  0.103511  0.103511   37363200.0        0.0   
7  0.107879  0.108315  0.107879  0.107879   46950400.0        0.0   
8  0.113556  0.113993  0.113556  0.113556   48003200.0        0.0   
9  0.124038  0.124475  0.124038  0.124038   55574400.0        0.0   

   Stock Splits Symbol  Timestamp  
0           0.0   AAPL 1980-12-12  
1           0.0   AAPL 1980-12-15  
2           0.0   AAPL 1980-12-16  
3           0.0   AAPL 1980-12-17  
4           0.0   AAPL 1980-12-18  
5 

C:\Users\Eric\AppData\Local\Temp/ipykernel_9680/3402867688.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(df[df.isna().any(axis=1)].sum())


Open            False
High            False
Low             False
Close           False
Volume          False
Dividends       False
Stock Splits    False
Symbol          False
Timestamp       False
dtype: bool


In [35]:
print(oneStockAAPL)

       Timestamp        Open        High         Low       Close     Volume  \
0     1980-12-12    0.100453    0.100890    0.100453    0.100453  469033600   
1     1980-12-15    0.095649    0.095649    0.095213    0.095213  175884800   
2     1980-12-16    0.088661    0.088661    0.088224    0.088224  105728000   
3     1980-12-17    0.090408    0.090845    0.090408    0.090408   86441600   
4     1980-12-18    0.093029    0.093466    0.093029    0.093029   73449600   
...          ...         ...         ...         ...         ...        ...   
10357 2022-01-10  169.080002  172.500000  168.169998  172.190002  106765600   
10358 2022-01-11  172.320007  175.179993  170.820007  175.080002   76138300   
10359 2022-01-12  176.119995  177.179993  174.820007  175.529999   74805200   
10360 2022-01-13  175.779999  176.619995  171.789993  172.190002   84505800   
10361 2022-01-14  171.339996  173.779999  171.089996  173.070007   80355000   

       Dividends  Stock Splits Symbol  
0          

In [36]:
# Get earliest timestamp for all stocks...

print(min(oneStockAAPL['Timestamp']))
print(max(oneStockAAPL['Timestamp']))
printDataframe(oneStockAAPL)

1980-12-12 00:00:00
2022-01-14 00:00:00
-----DATAFRAME-------
   Timestamp      Open      High       Low     Close     Volume  Dividends  \
0 1980-12-12  0.100453  0.100890  0.100453  0.100453  469033600        0.0   
1 1980-12-15  0.095649  0.095649  0.095213  0.095213  175884800        0.0   
2 1980-12-16  0.088661  0.088661  0.088224  0.088224  105728000        0.0   
3 1980-12-17  0.090408  0.090845  0.090408  0.090408   86441600        0.0   
4 1980-12-18  0.093029  0.093466  0.093029  0.093029   73449600        0.0   
5 1980-12-19  0.098707  0.099143  0.098707  0.098707   48630400        0.0   
6 1980-12-22  0.103511  0.103948  0.103511  0.103511   37363200        0.0   
7 1980-12-23  0.107879  0.108315  0.107879  0.107879   46950400        0.0   
8 1980-12-24  0.113556  0.113993  0.113556  0.113556   48003200        0.0   
9 1980-12-26  0.124038  0.124475  0.124038  0.124038   55574400        0.0   

   Stock Splits Symbol  
0           0.0   AAPL  
1           0.0   AAPL  
2   

In [37]:
oneStockAAPL.to_csv(r'../oneStockAAPL.csv', sep= ",", index = False)